In [1]:
## Load the model

In [2]:
import sys
import ast
sys.path.append("..")
sys.path.append("../src")

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import segmentation_models as sm
sm.set_framework('keras')
import ad_seg_utils as seg_utils
import bb_eval_utils as eval_utils
import csv
import math
import albumentations as A

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [3]:
SEGMENTATION = "AD"
PERTURBATION = "base"
print("Program initiating... type of segmentation: " + SEGMENTATION + ", type of perturbation: " + PERTURBATION)

BIN_SEG = False
CLASSES = ['background', 'skin']
WEIGHTS = np.array([1, 1])
target_idx = 1
MODEL_NAME = '/skin_base/base_skin_LR1.h5'
BACKBONE = 'efficientnetb3'
preprocess_input = sm.get_preprocessing(BACKBONE)

# set parameters based on the type of segmentation
if SEGMENTATION == 'SKIN' or SEGMENTATION == 'skin':
    pass
elif SEGMENTATION == 'AD' or SEGMENTATION == 'ad':
    BIN_SEG = False
    target_idx = 2
    CLASSES = ['background', 'skin', 'eczema']
    WEIGHTS = np.array([1, 1, 1])
    MODEL_NAME = '/AD_base/base_AD_LR1.h5'
else:
    print('Unexpected type of segmentation, should be either skin or ad\n program terminated')



"""# Model Evaluation"""
PROJ_DIR = "/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master"
PRED_DIR = os.path.join(PROJ_DIR, 'output/predictions/' + "test")
BB_DIR = os.path.join(PROJ_DIR, 'output/bounding_boxes')
EVAL_DIR = os.path.join(PROJ_DIR, 'output/evaluations')
MODEL_DIR = os.path.join(PROJ_DIR, 'output/models')
DATA_DIR = os.path.join(PROJ_DIR, 'data')

# new dataset
x_test_dir = os.path.join(DATA_DIR, 'test_set/reals')
y_test_dir = os.path.join(DATA_DIR, 'test_set/labels')

print('reading test images from: ' + str(x_test_dir))
print('reading test masks from: ' + str(y_test_dir))

test_dataset = seg_utils.Dataset(
    x_test_dir, 
    y_test_dir, 
    classes=CLASSES, 
    augmentation=None,
    preprocessing=seg_utils.get_preprocessing(preprocess_input),
    is_train=False,
    use_full_resolution=False,
    binary_seg=BIN_SEG,
)
eczema_dataset = seg_utils.Dataset(
    x_test_dir, 
    y_test_dir, 
    classes=['background', 'skin', 'eczema'],
    augmentation=None,
    preprocessing=seg_utils.get_preprocessing(preprocess_input),
    is_train=False,
    use_full_resolution=False,
    binary_seg=0,
)


model = seg_utils.load_model(dir=MODEL_DIR+MODEL_NAME, classes=CLASSES, weights=WEIGHTS)
print('Trained model loaded!')

Program initiating... type of segmentation: AD, type of perturbation: base
reading test images from: /rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/test_set/reals
reading test masks from: /rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/test_set/labels
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2022-06-02 15:54:49.966968: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-06-02 15:54:49.986682: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-02 15:54:49.986739: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: cx1-106-6-4.cx1.hpc.ic.ac.uk
2022-06-02 15:54:49.986751: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: cx1-106-6-4.cx1.hpc.ic.ac.uk
2022-06-02 15:54:49.986839: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.60.2
2022-06-02 15:54:49.986872: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.60.2
2022-06-02 15:54:49.986881: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.60.2
2022-06-02 15:54:49.988491

Trained model loaded!


## Example to predict mask, generate box and produce square crops

In [4]:
# %%time
# folder_dir = '/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/test_set/reals'
folder_dir = '/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/TLA4AE_origin'
fns = os.listdir(folder_dir)

for file in fns:
    print(os.path.join(folder_dir, file))
    seg_utils.run_sigle_pred(model, os.path.join(folder_dir, file), "/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/output/predictions/test", file, target_idx=target_idx, resize_ratio=0.4, preprocessing=seg_utils.get_preprocessing(preprocess_input), refno="0000",visno="0")


/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/TLA4AE_origin/97a217c4-5c8e-425b-95a6-31763e429d7b.png
directory created!


sh: which: line 1: syntax error: unexpected end of file
sh: error importing function definition for `which'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: scl: line 1: syntax error: unexpected end of file
sh: error importing function definition for `scl'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: which: line 1: syntax error: unexpected end of file
sh: error importing function definition for `which'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: scl: line 1: syntax error: unexpected end of file
sh: error importing function definition for `scl'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: which: line 1: syntax error: unexpected end of file
sh: error importing function definition for `which'
sh: module: line 1: syntax error: unexpected

2022-06-02 16:02:45.358523: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 452984832 exceeds 10% of system memory.
2022-06-02 16:02:45.399879: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 452984832 exceeds 10% of system memory.
2022-06-02 16:02:45.399879: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 452984832 exceeds 10% of system memory.
2022-06-02 16:02:45.505653: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 452984832 exceeds 10% of system memory.
2022-06-02 16:02:45.900878: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 150994944 exceeds 10% of system memory.


97a217c4-5c8e-425b-95a6-31763e429d7b.png : cropping done!
/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/TLA4AE_origin/33cdf83c-d178-401c-81af-eecbec6eaff8.JPG
directory created!
33cdf83c-d178-401c-81af-eecbec6eaff8.JPG : cropping done!
/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/TLA4AE_origin/45a71f24-2d0a-48c7-bba7-c3de4c56e551.png
directory created!
45a71f24-2d0a-48c7-bba7-c3de4c56e551.png : cropping done!
/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/TLA4AE_origin/488688d9-b7ce-4aa0-84c3-7c55e992518e.JPG
directory created!
488688d9-b7ce-4aa0-84c3-7c55e992518e.JPG : cropping done!
/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/TLA4AE_origin/3a2f119c-5b5d-4c32-b2e8-a7d0c51a2733.jpg
directory created!
3a2f119c-5b5d-4c32-b2e8-a7d0c51a2733.jpg : cropping done!
/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/TLA4AE_origin/de676a04-9977-4